In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.utils.data import DataLoader, Dataset

In [2]:
battles = pd.read_csv("C:/Users/coliv/summerProjects/Summer-Repository/massiveBatleDF.csv")
battles

,Unnamed: 0,level_0,turn,winner,party1,party2,index,id,elo,ActivePokemon1,...,defense1,defense2,spattack1,spattack2,spdefense1,spdefense2,speed1,speed2,movePool1,movePool2
0,0,1,1,p1a,"Tyranitar, Skarmory, Aerodactyl, Gengar, Aggro...","Tyranitar, Skarmory, Blissey, Claydol, Milotic...",0,gen3ou-2178354621,1495.0,Tyranitar,...,0,0,0,0,0,0,0,0,"['switch', 'Earthquake', 'Dragon Dance', 'Rock...","['switch', 'Rock Slide']"
1,1,2,2,p1a,"Tyranitar, Skarmory, Aerodactyl, Gengar, Aggro...","Tyranitar, Skarmory, Blissey, Claydol, Milotic...",0,gen3ou-2178354621,1495.0,Tyranitar,...,0,0,0,0,0,0,0,0,"['switch', 'Earthquake', 'Dragon Dance', 'Rock...","['switch', 'Spikes', 'Roar']"
2,2,3,3,p1a,"Tyranitar, Skarmory, Aerodactyl, Gengar, Aggro...","Tyranitar, Skarmory, Blissey, Claydol, Milotic...",0,gen3ou-2178354621,1495.0,Skarmory,...,0,0,0,0,0,0,0,0,"['Roar', 'switch', 'Drill Peck', 'Spikes', 'Pr...","['switch', 'Spikes', 'Roar']"
3,3,4,4,p1a,"Tyranitar, Skarmory, Aerodactyl, Gengar, Aggro...","Tyranitar, Skarmory, Blissey, Claydol, Milotic...",0,gen3ou-2178354621,1495.0,Aerodactyl,...,0,0,0,0,0,0,0,0,"['switch', 'Rock Slide']","['switch', 'Spikes', 'Roar']"
4,4,5,5,p1a,"Tyranitar, Skarmory, Aerodactyl, Gengar, Aggro...","Tyranitar, Skarmory, Blissey, Claydol, Milotic...",0,gen3ou-2178354621,1495.0,Gengar,...,0,0,0,0,0,0,0,0,"['switch', 'Ice Punch', 'Giga Drain', 'Will-O-...","['switch', 'Spikes', 'Roar']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995746,32,33,33,p2a,"Tyranitar, Swampert, Aerodactyl, Starmie, Blis...","Scizor, Camerupt, Celebi, Magneton, Gyarados, ...",0,gen3ou-2026201285,1263.0,Blissey,...,0,0,0,0,0,0,0,0,"['switch', 'Ice Beam', 'Toxic', 'Seismic Toss']","['Hidden Power', 'switch', 'Dragon Dance', 'Do..."
995747,33,34,34,p2a,"Tyranitar, Swampert, Aerodactyl, Starmie, Blis...","Scizor, Camerupt, Celebi, Magneton, Gyarados, ...",0,gen3ou-2026201285,1263.0,Tyranitar,...,0,0,0,0,0,0,0,0,"['switch', 'Fire Blast']","['Hidden Power', 'switch', 'Dragon Dance', 'Do..."
995748,34,35,35,p2a,"Tyranitar, Swampert, Aerodactyl, Starmie, Blis...","Scizor, Camerupt, Celebi, Magneton, Gyarados, ...",0,gen3ou-2026201285,1263.0,Swampert,...,0,0,0,0,0,0,0,0,"['switch', 'Hydro Pump', 'Ice Beam', 'Protect'...","['Hidden Power', 'switch', 'Dragon Dance', 'Do..."
995749,35,36,36,p2a,"Tyranitar, Swampert, Aerodactyl, Starmie, Blis...","Scizor, Camerupt, Celebi, Magneton, Gyarados, ...",0,gen3ou-2026201285,1263.0,Swampert,...,0,0,0,0,0,0,0,0,"['switch', 'Hydro Pump', 'Ice Beam', 'Protect'...","['Hidden Power', 'switch', 'Dragon Dance', 'Do..."


Come up with an algorithm that will find the most common set on each pokemon. 

Things that you need to watch out for:

    1) You need to be careful of variable battle length. Longer battles will contribute more than shorter battles
        a) Consider sampling a constant number of turns from each battle. 
    
    2) You need to be able to catch every pokemon in the dataset. This will lead to inherent bias. However, since only biases are being added, I guess it means the model will just know pokemon that it has the most experience with better.
        a) It will be a challenge determining how biases should be scaled. Maybe, find a better way to differentiate. 


    3) Could I accomplish the same thing by feeding each output through a multilayer perceptron?

In [3]:

battleLengths = pd.DataFrame(battles[battles["winner"] != "tie"].groupby(["id"])["id"].count())
battleLenArray = battles[battles["winner"] != "tie"].groupby(["id"])["id"].count().to_numpy()
ids = battleLengths.index.to_numpy()

LenDF = pd.DataFrame({"length" : battleLenArray})
idDF = pd.DataFrame({"id" : ids})
battleLenDF = pd.concat([idDF, LenDF], axis = 1)

battlesLongCut = battleLenDF[battleLenDF["length"] < 100]

battles = battles.loc[battles["id"].isin(battlesLongCut["id"])].drop("level_0", axis = 1).drop("Unnamed: 0", axis = 1).reset_index()

In [4]:
battles

,level_0,turn,winner,party1,party2,index,id,elo,ActivePokemon1,ActivePokemon2,...,defense1,defense2,spattack1,spattack2,spdefense1,spdefense2,speed1,speed2,movePool1,movePool2
0,0,1,p1a,"Tyranitar, Skarmory, Aerodactyl, Gengar, Aggro...","Tyranitar, Skarmory, Blissey, Claydol, Milotic...",0,gen3ou-2178354621,1495.0,Tyranitar,Tyranitar,...,0,0,0,0,0,0,0,0,"['switch', 'Earthquake', 'Dragon Dance', 'Rock...","['switch', 'Rock Slide']"
1,1,2,p1a,"Tyranitar, Skarmory, Aerodactyl, Gengar, Aggro...","Tyranitar, Skarmory, Blissey, Claydol, Milotic...",0,gen3ou-2178354621,1495.0,Tyranitar,Skarmory,...,0,0,0,0,0,0,0,0,"['switch', 'Earthquake', 'Dragon Dance', 'Rock...","['switch', 'Spikes', 'Roar']"
2,2,3,p1a,"Tyranitar, Skarmory, Aerodactyl, Gengar, Aggro...","Tyranitar, Skarmory, Blissey, Claydol, Milotic...",0,gen3ou-2178354621,1495.0,Skarmory,Skarmory,...,0,0,0,0,0,0,0,0,"['Roar', 'switch', 'Drill Peck', 'Spikes', 'Pr...","['switch', 'Spikes', 'Roar']"
3,3,4,p1a,"Tyranitar, Skarmory, Aerodactyl, Gengar, Aggro...","Tyranitar, Skarmory, Blissey, Claydol, Milotic...",0,gen3ou-2178354621,1495.0,Aerodactyl,Skarmory,...,0,0,0,0,0,0,0,0,"['switch', 'Rock Slide']","['switch', 'Spikes', 'Roar']"
4,4,5,p1a,"Tyranitar, Skarmory, Aerodactyl, Gengar, Aggro...","Tyranitar, Skarmory, Blissey, Claydol, Milotic...",0,gen3ou-2178354621,1495.0,Gengar,Skarmory,...,0,0,0,0,0,0,0,0,"['switch', 'Ice Punch', 'Giga Drain', 'Will-O-...","['switch', 'Spikes', 'Roar']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
901459,995746,33,p2a,"Tyranitar, Swampert, Aerodactyl, Starmie, Blis...","Scizor, Camerupt, Celebi, Magneton, Gyarados, ...",0,gen3ou-2026201285,1263.0,Blissey,Gyarados,...,0,0,0,0,0,0,0,0,"['switch', 'Ice Beam', 'Toxic', 'Seismic Toss']","['Hidden Power', 'switch', 'Dragon Dance', 'Do..."
901460,995747,34,p2a,"Tyranitar, Swampert, Aerodactyl, Starmie, Blis...","Scizor, Camerupt, Celebi, Magneton, Gyarados, ...",0,gen3ou-2026201285,1263.0,Tyranitar,Gyarados,...,0,0,0,0,0,0,0,0,"['switch', 'Fire Blast']","['Hidden Power', 'switch', 'Dragon Dance', 'Do..."
901461,995748,35,p2a,"Tyranitar, Swampert, Aerodactyl, Starmie, Blis...","Scizor, Camerupt, Celebi, Magneton, Gyarados, ...",0,gen3ou-2026201285,1263.0,Swampert,Gyarados,...,0,0,0,0,0,0,0,0,"['switch', 'Hydro Pump', 'Ice Beam', 'Protect'...","['Hidden Power', 'switch', 'Dragon Dance', 'Do..."
901462,995749,36,p2a,"Tyranitar, Swampert, Aerodactyl, Starmie, Blis...","Scizor, Camerupt, Celebi, Magneton, Gyarados, ...",0,gen3ou-2026201285,1263.0,Swampert,Gyarados,...,0,0,0,0,0,0,0,0,"['switch', 'Hydro Pump', 'Ice Beam', 'Protect'...","['Hidden Power', 'switch', 'Dragon Dance', 'Do..."


In [5]:
trainingIndeces = battles.groupby(["id"])["id"].count().index[:28000]
validationIndeces = battles.groupby(["id"])["id"].count().index[28000:]

trainingBattles = battles[battles["id"].isin(trainingIndeces)]

In [6]:
df = trainingBattles[["id", "ActivePokemon1", "ActivePokemon2", "move1", "move2"]]
df["move1"] = df["move1"].str.lower().str.replace(" ", "-")
df["move2"] = df["move2"].str.lower().str.replace(" ", "-")
df["ActivePokemon1"] = df["ActivePokemon1"].str.lower()
df["ActivePokemon2"] = df["ActivePokemon2"].str.lower()

C:\Users\coliv\AppData\Local\Temp\ipykernel_24564\2185080675.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["move1"] = df["move1"].str.lower().str.replace(" ", "-")
C:\Users\coliv\AppData\Local\Temp\ipykernel_24564\2185080675.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["move2"] = df["move2"].str.lower().str.replace(" ", "-")
C:\Users\coliv\AppData\Local\Temp\ipykernel_24564\2185080675.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

In [7]:
df[df["ActivePokemon1"] == "bulbasaur"]

,id,ActivePokemon1,ActivePokemon2,move1,move2
188672,gen3ou-2148321532,bulbasaur,blissey,sleep-powder,seismic-toss
188673,gen3ou-2148321532,bulbasaur,blissey,sleep-powder,switch
188674,gen3ou-2148321532,bulbasaur,jirachi,leech-seed,NaN
188675,gen3ou-2148321532,bulbasaur,jirachi,sunny-day,fire-punch
188676,gen3ou-2148321532,bulbasaur,jirachi,NaN,NaN
208140,gen3ou-2145366186,bulbasaur,snorlax,substitute,switch
208141,gen3ou-2145366186,bulbasaur,salamence,leech-seed,dragon-dance
208142,gen3ou-2145366186,bulbasaur,salamence,protect,aerial-ace
208143,gen3ou-2145366186,bulbasaur,salamence,substitute,aerial-ace
208144,gen3ou-2145366186,bulbasaur,salamence,protect,switch


In [8]:
moveDict = pd.read_csv("C:/Users/coliv/summerProjects/Summer-Repository/moveDict.csv").drop("Unnamed: 0", axis = 1)
moveDict = moveDict.T[moveDict.T[0] < 10000]
moveDict = moveDict.T
moveDict

,pound,karate-chop,double-slap,comet-punch,mega-punch,pay-day,fire-punch,ice-punch,thunder-punch,scratch,...,magical-leaf,water-sport,calm-mind,leaf-blade,dragon-dance,rock-blast,shock-wave,water-pulse,doom-desire,psycho-boost
0,1,2,3,4,5,6,7,8,9,10,...,345,346,347,348,349,350,351,352,353,354


In [9]:
df1 = df.set_index("ActivePokemon1")
df1 = df1[["id", "move1"]]

df2 = df.set_index("ActivePokemon2")
df2 = df2[["id", "move2"]]

In [10]:
df1

,id,move1
ActivePokemon1,,
forretress,gen3ou-2158252620,NaN
forretress,gen3ou-2158252620,switch
blissey,gen3ou-2158252620,toxic
blissey,gen3ou-2158252620,heal-bell
blissey,gen3ou-2158252620,switch
...,...,...
blissey,gen3ou-2026201285,switch
tyranitar,gen3ou-2026201285,switch
swampert,gen3ou-2026201285,ice-beam


In [11]:
df1 = df1[df1["move1"] != "switch"].dropna()
df2 = df2[df2["move2"] != "switch"].dropna()


In [12]:
monDict = pd.read_csv("C:/Users/coliv/summerProjects/Summer-Repository/monDict.csv").drop("Unnamed: 0", axis = 1)
mons = monDict.columns

moveSetDistributions = {}
for i in mons :
    moveSetDistributions.update({i : np.zeros(354)})



In [13]:
moveSetDistributions.get("charmeleon")

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

You need to change the bottom so that the test set is not included. Just take the first however many rows are in your test set because you did not shuffle at that point when preparing your tensors. 

In [14]:
def find_proportions(battleDF, moveSetDistributions, player_number) :
    for i in mons :
        try :
            currDF = battleDF.loc[i]
            currDF = currDF.drop_duplicates()
            for j in currDF[f"move{player_number}"] :
                idx = moveDict[j]
                moveSetDistributions.get(i)[idx] += 1
        except:
            continue

In [15]:
moveSetDistributions.get("charmeleon")

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [16]:
find_proportions(df1, moveSetDistributions, 1)
find_proportions(df2, moveSetDistributions, 2)

In [17]:
moveSetDistributions.get("jolteon")

array([0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       3.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 4.160e+02, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 2.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       1.000e+00, 0.000e+00, 3.800e+01, 0.000e+00, 6.460e+02, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 

In [18]:
moveSetDistributionsDF = pd.DataFrame(moveSetDistributions)
moveSetDistributionsDF

,bulbasaur,ivysaur,venusaur,charmander,charmeleon,charizard,squirtle,wartortle,blastoise,caterpie,...,regirock,regice,registeel,latias,latios,kyogre,groudon,rayquaza,jirachi,deoxys
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,0.0,0.0,0.0,0.0,0.0,80.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
350,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
351,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
352,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,0.0


In [19]:
moveSetDict = pd.read_csv("C:/Users/coliv/summerProjects/Summer-Repository/moveset_dictionary.csv").drop("Unnamed: 0", axis = 1).set_index("name").T
moveSetDict


for i in mons :
    arr = moveSetDistributions.get(i)
    totalActions = np.sum(arr)
    if totalActions == 0 :
        continue
    for j in range(len(arr)) :
        arr[j] = arr[j] / totalActions

In [20]:
moveSetDistributionsDF = pd.DataFrame(moveSetDistributions)
moveSetDistributionsDF

,bulbasaur,ivysaur,venusaur,charmander,charmeleon,charizard,squirtle,wartortle,blastoise,caterpie,...,regirock,regice,registeel,latias,latios,kyogre,groudon,rayquaza,jirachi,deoxys
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000074,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,0.0,0.0,0.0,0.0,0.0,0.011852,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
350,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
351,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
352,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.009873,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001926,0.0


In [58]:
max(moveSetDistributionsDF["regirock"])

0.24751491053677932